In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import sqlalchemy
import os
import json

from dotenv import load_dotenv
load_dotenv()

import matplotlib as plt
%matplotlib inline

In [2]:
conn = psycopg2.connect(
    user=os.getenv("POSTGRES_USER"), 
    password=os.getenv("POSTGRES_PASSWORD"), 
    host="127.0.0.1", 
    port="54320", 
    database="dcat")

In [3]:
cur = conn.cursor()

In [4]:
print ( conn.get_dsn_parameters(),"\n")

{'user': 'dcat', 'dbname': 'dcat', 'host': '127.0.0.1', 'port': '54320', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [5]:
cur.execute("SELECT version();")
record = cur.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 12.0 (Debian 12.0-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 



# Init

### Relationship description/attributes

In [6]:
cur.execute("delete from tbl_er_desc")

### Relationships/Edges 

In [7]:
cur.execute("delete from tbl_er")

### Node details table

In [8]:
cur.execute("delete from tbl_detail")

# Get data

In [9]:
xls = pd.ExcelFile("../data/testdata.xlsx")

In [10]:
xls.sheet_names

['Formål GML',
 'Formål',
 'Ark1',
 'data-catalog-backend',
 'data-catalog-policies',
 'Category']

In [23]:
conn.commit()

# Populate tables

### Relationships

In [12]:
relationships = ["has purpose", "based on purpose", "has attribute", "attribute of"]
index = 1
for relationship in relationships:
    insert_query = f"insert into tbl_er_desc values ({index},'{relationship}')"
    cur.execute(insert_query)
    index += 1
    
conn.commit()

In [13]:
df = sqlio.read_sql_query("select * from tbl_er_desc limit 2", conn)
df

,id,info
0,1,has purpose
1,2,based on purpose


### Processing activities

In [14]:
sheet=xls.sheet_names[1]
df_formål = pd.read_excel(xls, sheet, encoding='latin1')
df_formål = df_formål[['Forslag til endelig formålskode', 'Formålsbeskrivelse']]
df_formål.columns=['code', 'description']
df_formål['name'] = df_formål['code'].apply(lambda x: x.upper())
df_formål = df_formål.append({'name': 'NOT AVAILABLE', 'description': 'Not Available -Verdi mangler'}, ignore_index=True)
df_formål.drop(['code'], axis=1, inplace=True)
df_formål['type'] = 'prosessing activity'
df_formål.tail()

,description,name,type
33,Behandle og vurdere rett til uføretrygd som sk...,UFØRETRYGD,prosessing activity
34,Behandle og vurdere rett til ventelønn som ska...,VENTELØNN,prosessing activity
35,Behandle og vurdere rett til ytelser ved yrkes...,YRKESSKADE- OG SYKDOM,prosessing activity
36,Behandle og vurdere rett til ytelser til famil...,YTELSER TIL FAMILIEPLEIER,prosessing activity
37,Not Available -Verdi mangler,NOT AVAILABLE,prosessing activity


In [15]:
for index, row in df_formål.iterrows():
    id = index + 1
    jsonb = row.to_json()
    insert_query = f"insert into tbl_detail values ({id},'{jsonb}', now())"
    cur.execute(insert_query, (jsonb,))
    
conn.commit()

### Legal basis

In [17]:
sheet=xls.sheet_names[4]

cur.execute("select max(id) from tbl_detail")
index = cur.fetchone()[0]

df = pd.read_excel(xls, sheet, encoding='latin1')
legal_basis_list = df['legalBasisDescription'].unique()
for legal_basis in legal_basis_list:
    index = index + 1
    jsonb = json.dumps('{"name":' + legal_basis + ', "type": "legal_basis"}')
    insert_query = f"insert into tbl_detail values ({index},'{jsonb}', now())"
    cur.execute(insert_query, (index, jsonb,))
    
conn.commit()

### Attributes

In [24]:
sheet=xls.sheet_names[3]

cur.execute("select max(id) from tbl_detail")
index = cur.fetchone()[0]

df_attributes = pd.read_excel(xls, sheet, encoding='latin1')
df_attributes = df_attributes[['title', 'description']]
df_attributes.columns=['name', 'description']
df_attributes = df_attributes.append({'name': 'NOT AVAILABLE', 'description': 'Not Available -Verdi mangler'}, ignore_index=True)
df_attributes['type'] = 'attribute'
df_attributes.head()

,name,description,type
0,Sivilstand,En overordnet kategori som beskriver en person...,attribute
1,Arbeidsforhold,"Avtaleforhold hvor den ene part, arbeidstakere...",attribute
2,Kjønn,TODO - mangler i begrepskatalogen og i MFNs be...,attribute
3,Navn,I Norge skal alle ha fornavn og ett enkelt ell...,attribute
4,Fødselsdato,Datoen personen er født.,attribute


In [25]:
for i, row in df_attributes.iterrows():
    index = index + 1
    jsonb = row.to_json()
    insert_query = f"insert into tbl_detail values ({index},'{jsonb}', now())"
    cur.execute(insert_query, (jsonb,))
    
conn.commit()

### test

In [26]:
df = sqlio.read_sql_query("select * from tbl_detail", conn)
df.tail()

,id,info,crt_time
214,215,"{'name': 'Flyktningstatus ', 'type': 'attribut...",2019-10-07 17:45:40.436798
215,216,"{'name': 'Foreldreansvar', 'type': 'attribute'...",2019-10-07 17:45:40.436798
216,217,"{'name': 'Forsterforeldre ', 'type': 'attribut...",2019-10-07 17:45:40.436798
217,218,"{'name': 'Fullmakt ', 'type': 'attribute', 'de...",2019-10-07 17:45:40.436798
218,219,"{'name': 'NOT AVAILABLE', 'type': 'attribute',...",2019-10-07 17:45:40.436798


### Edges

In [27]:
cur.execute("insert into tbl_er values (1,2,array[1],now());")
cur.execute("insert into tbl_er values (2,1,array[2],now());")
cur.execute("insert into tbl_er values (1,3,array[1],now());")
cur.execute("insert into tbl_er values (3,1,array[2],now());")
cur.execute("insert into tbl_er values (5,2,array[1],now());")
cur.execute("insert into tbl_er values (2,5,array[2],now());")
cur.execute("insert into tbl_er values (5,3,array[1],now());")
cur.execute("insert into tbl_er values (3,5,array[2],now());")
cur.execute("insert into tbl_er values (4,1,array[1],now());")
cur.execute("insert into tbl_er values (1,4,array[2],now());")
cur.execute("insert into tbl_er values (6,5,array[1],now());")
cur.execute("insert into tbl_er values (5,6,array[2],now());")
cur.execute("insert into tbl_er values (7,1,array[1],now());")
cur.execute("insert into tbl_er values (1,7,array[2],now());")
cur.execute("insert into tbl_er values (8,5,array[1],now());")
cur.execute("insert into tbl_er values (5,8,array[2],now());")

In [28]:
conn.commit()

In [29]:
sql = """
create or replace function graph_search1(      
  IN i_root int,                       -- The node that the search is based on        
  IN i_depth int  default 99999,       -- the tier to search (the depth limit)      
  IN i_limit int8 default 2000000000,  -- limit the number of records returned for each tier      
  OUT o_path int[],                    -- output: path, an array of IDs      
  OUT o_point1 int,                    -- output: point 1 ID      
  OUT o_point2 int,                    -- output: point 2 ID      
  OUT o_link_prop int2[],              -- output: the connection property between the two current points      
  OUT o_link_prop_all text,            -- output: the connection property from the starting node to the current node      
  OUT o_depth int                      -- output: current depth (tier)      
) returns setof record as 
$$
      
declare      
  sql text;      
begin      
sql := format($_$      
WITH RECURSIVE search_graph(        
  c1,     -- point 1        
  c2,     -- point 2        
  prop,   -- current edge property      
  all_prop,  -- properties of all edges  
  depth,  -- current depth, starting from 1         
  path    -- path, stored as an array         
) AS (        
        select c1,c2,prop,all_prop,depth,path from (        
        SELECT                               -- ROOT node query        
          g.c1,                              -- point 1        
          g.c2,                              -- point 2        
          g.prop,                            -- edge property        
      g.prop::text as all_prop,          -- properties of all edges  
          1 depth,                           -- initial depth=1        
          ARRAY[g.c1, g.c2] path             -- initial path        
        FROM tbl_er AS g         
        WHERE         
          c1 = %s                            -- ROOT node=?        
          limit %s                           -- How many records are limited at each tier?        
        ) t        
      UNION ALL        
        select c1,c2,prop,all_prop,depth,path from (        
        SELECT                               -- recursive clause         
          g.c1,                              -- point 1        
          g.c2,                              -- point 2        
          g.prop,                            -- edge property     
      sg.all_prop || g.prop::text as all_prop,    -- properties of all edges  
          sg.depth + 1 depth,                   -- depth +1        
          sg.path || g.c2 path                 -- Add a new point to the path        
        FROM tbl_er AS g, search_graph AS sg    -- circular INNER JOIN        
        WHERE         
          g.c1 = sg.c2                       -- recursive JOIN condition        
          AND (g.c2 <> ALL(sg.path))                      -- Prevent loop, determine whether it is a loop and judge if the new point is already in the previous path   
          AND sg.depth <= %s                 -- search depth =?          
          limit %s                           -- How many records are limited at each tier?       
        ) t        
)        
SELECT path as o_path, c1 as o_point1, c2 as o_point2, prop as o_link_prop, all_prop as o_link_prop_all, depth as o_depth      
FROM search_graph;                           -- query a recursive table. You can add LIMIT output or use a cursor       
$_$, i_root, i_limit, i_depth, i_limit      
);      
      
return query execute sql;      
      
end;      

$$
 language plpgsql strict;   
"""

cur.execute(sql)

In [30]:
conn.commit()

In [31]:
%%time
sql = "select * from graph_search1(1)"
df = sqlio.read_sql_query(sql, conn)

CPU times: user 2.55 ms, sys: 1.57 ms, total: 4.11 ms
Wall time: 12.6 ms


In [34]:
sql = "select * from graph_search1(1)"
df = sqlio.read_sql_query(sql, conn)

In [35]:
df.head()

,o_path,o_point1,o_point2,o_link_prop,o_link_prop_all,o_depth
0,"[1, 2]",1,2,[1],{1},1
1,"[1, 3]",1,3,[1],{1},1
2,"[1, 4]",1,4,[2],{2},1
3,"[1, 7]",1,7,[2],{2},1
4,"[1, 2, 5]",2,5,[2],{1}{2},2


In [36]:
df_links = sqlio.read_sql_query("select * from tbl_er_desc", conn)
df_links.head()

,id,info
0,1,has purpose
1,2,based on purpose
2,3,has attribute
3,4,attribute of


In [41]:
df_nodes = sqlio.read_sql_query("select * from tbl_detail", conn)
#df_nodes['name'] = df_nodes['info'].apply(lambda x : x['name'])
#df_nodes[['id', 'name']].head()

In [44]:
df_nodes.head(500)

,id,info,crt_time
0,1,"{'name': 'ALDERSPENSJON', 'type': 'prosessing ...",2019-10-07 17:43:49.778671
1,2,"{'name': 'ARBEIDSAVKLARINGSPENGER (AAP)', 'typ...",2019-10-07 17:43:49.778671
2,3,{'name': 'AVTALEFESTET PENSJON (AFP) FOR PRIVA...,2019-10-07 17:43:49.778671
3,4,{'name': 'AVTALEFESTET PENSJON (AFP) FOR STATL...,2019-10-07 17:43:49.778671
4,5,"{'name': 'BARNEBIDRAG', 'type': 'prosessing ac...",2019-10-07 17:43:49.778671
5,6,"{'name': 'BARNETRYGD', 'type': 'prosessing act...",2019-10-07 17:43:49.778671
6,7,"{'name': 'BIDRAGSFORSKUDD', 'type': 'prosessin...",2019-10-07 17:43:49.778671
7,8,"{'name': 'DAGPENGER', 'type': 'prosessing acti...",2019-10-07 17:43:49.778671
8,9,"{'name': 'EKTEFELLEBIDRAG', 'type': 'prosessin...",2019-10-07 17:43:49.778671
9,10,"{'name': 'ETTERLATTEPENSJON OG BARNEPENSJON', ...",2019-10-07 17:43:49.778671


In [27]:
df_edgelist=df[['o_point1','o_point2','o_link_prop','o_depth']].copy()
df_edgelist.columns=['source','target','id','weight']
df_edgelist['id'] = df_edgelist['id'].apply(lambda x: x[0])
df_edgelist = pd.merge(df_edgelist, df_links, on='id', how='left')
df_edgelist = pd.merge(df_edgelist, df_nodes[['id', 'name']], left_on='source', right_on='id')
df_edgelist = pd.merge(df_edgelist, df_nodes[['id', 'name']], left_on='target', right_on='id')
#df_edgelist = df_edgelist[['info', 'name_x', 'name_y','id','weight']]
#df_edgelist.columns = ['name', 'source', 'target','id','weight']
df_edgelist.head()

KeyError: "None of [Index(['o_point1', 'o_point2', 'o_link_prop', 'o_depth'], dtype='object')] are in the [columns]"

In [ ]:
import altair as alt
alt.renderers.enable('notebook')
alt.themes.enable('opaque')
import networkx as nx
import nx_altair as nxa
import numpy as np

G = nx.from_pandas_edgelist(df_edgelist, 'source', 'target', ['name','id','weight'])
pos = nx.spring_layout(G)

In [ ]:
nx.draw_networkx(
    G=G,
    pos=pos
)

In [ ]:
# Compute positions for viz.
pos = nx.spring_layout(G)

viz = nxa.draw_networkx(
    G, pos=pos,
    node_color='id:N',
    cmap='viridis',
    width='weight:N',
    edge_color='black',
    node_tooltip=['weight:N']
)

viz.interactive()